In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

In [14]:
# Step 1: Set the path to your kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = r"C:\Users\cnuna\.kaggle"  # adjust if needed

# Step 2: Authenticate Kaggle API
api = KaggleApi()
api.authenticate()

# Step 3: Download dataset (replace with the Kaggle dataset API path and do NOT include CSV filename)
dataset = "rjmanoj/credit-card-customer-churn-prediction"

# Step 4: Download and unzip all files to current folder
api.dataset_download_files(dataset, path=".", unzip=True)

# Step 5: Load CSV into Pandas
df = pd.read_csv("Churn_Modelling.csv")
df.head(5)

Dataset URL: https://www.kaggle.com/datasets/rjmanoj/credit-card-customer-churn-prediction


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
